In [4]:
import os
import json
import pandas as pd

from tabulate import tabulate

In [5]:
def json_from_file(filename): # assumes the file is in JSON format
    with open(filename) as f: data = json.loads(f.read())
    return data

def no_dupes(l): # abstract ou[ d['metadata']['benchmark'] for d in data ]t the whole weird list/set method of removing duplicates
    return list(set(l))

In [6]:
data_directory = "/home/alejandro/jRAPL/tests/dacapo/async-monitors/results/jolteon-results"
os.chdir(data_directory)

In [7]:
filenames = [ _ for _ in os.listdir() if _.endswith('.aggregate-stats.json') ]
data = [ json_from_file(filename) for filename in filenames ]
display(data[0])

{'metadata': {'samplingRate': 1,
  'lifetime': 3582.25,
  'numSamples': 3278.75,
  'benchmark': 'tomcat',
  'iteration': 'AGGREGATE',
  'monitor_type': 'c-dynamicarray'},
 'memory': {'jraplon': {'avg': 170774395.479221,
   'stdev': 172333165.04320914,
   'global_min': 465019760,
   'global_max': 18953384,
   'avg_min': 478279668,
   'avg_max': 17897346,
   'median_median': 145086648.0,
   'average_median': 144467450.0},
  'jraploff': {'avg': 152544363.90966853,
   'stdev': 152620262.56520727,
   'global_min': 467846424,
   'global_max': 18876360,
   'avg_min': 481126746,
   'avg_max': 17840994,
   'median_median': 119934372.0,
   'average_median': 119473442.0}},
 'persocket': {'1': {'dram': {'energy-per-sample': {'avg': 0.0008819540843573255,
     'stdev': 0.0011131012350601853,
     'num_samples': 3277.75}},
   'pkg': {'energy-per-sample': {'avg': 0.0213188391427078,
     'stdev': 0.007888219898613196,
     'num_samples': 3277.75}},
   'time-between-samples': {'avg': 1092.926016322172

In [72]:
#def percent_difference(a, b, round_to=5):
#    return round( ((a-b)/b)*100, round_to) 

def diff_tuple(a,b,r=4):
    diff = a-b
    percent_diff = (diff/b) * 100
    return  ( round(float(percent_diff),r) , round(float(diff),r) )

def memory_comparison(memdata):
    jraplon  = memdata['jraplon']
    jraploff = memdata['jraploff']
    res = {}

    """should i be subtracting stdev? its not in the current list, but if i add it as an item of the list then itll work in just fine"""

    for v in ['avg' , 'median_median' , 'average_median' , 'global_min' , 'global_max' , 'avg_min' , 'avg_max' ]:
        tup = diff_tuple(jraplon[v],jraploff[v])
        res[v] = tup #"(%.4f,  %.4f)" % (tup[0],tup[1])
    return res

def monitor_ranking(data):
    for v in ['avg' , 'median_median' , 'average_median' , 'global_min' , 'global_max' , 'avg_min' , 'avg_max' ]:
        #print([ data[monitor_type][v] for monitor_type in data ])
        for monitor_type in list(set(data)):
            print(monitor_type)
            #print(data[monitor_type])
    return 1111111111


In [73]:
benchmarks = no_dupes([ d['metadata']['benchmark'] for d in data ])
monitor_types = no_dupes([ d['metadata']['monitor_type'] for d in data ])

data_table = {}
#for monitor_type in monitor_types:
#    data_table[monitor_type] = {}
for d in data:
    metadata = d['metadata']
    monitor_type = metadata['monitor_type']
    benchmark = metadata['benchmark']
    if not benchmark in data_table:
        data_table[benchmark] = {}
    if not monitor_type in data_table[benchmark]:
        data_table[benchmark][monitor_type] = {}
    data_table[benchmark][monitor_type] = memory_comparison(d['memory'])
    data_table[benchmark]['monitor_rank'] = monitor_ranking(data_table[benchmark])


c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
java
java
java
java
java
java
java
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
java
java
java
java
java
java
java
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-dynamicarray
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
c-dynamicarray
monitor_rank
c-linklist
java
monitor_rank
c-linklist
java
monitor_rank
c-linklist
java
monitor_rank
c-linkl

In [17]:
os.mkdir('latex')
for benchmark in data_table:
    fname = "latex/"+benchmark+"_memory-comparison.tex"
    with open(fname,'w')as fh:
        df = pd.DataFrame.from_dict(data_table[benchmark])
        fh.write( tabulate(df, headers='keys',tablefmt='latex') )